# 朴素贝叶斯

## 贝叶斯理论&条件概率

### 贝叶斯理论
比较的是 $p(c_1|x, y)$ 和 $p(c_2|x, y)$，这些符号所代表的具体意义是：给定某个由 $x, y$ 表示的数据点，那么该数据点来自类别 $c_1$ 的概率是多少？数据点来自类别 $c_2$ 的概率又是多少？注意这些概率与概率 $p(x, y|c_1)$ 并不一样，不过可以使用贝叶斯准则来交换概率中条件与结果。具体地，应用贝叶斯准则得到：

$$
p(c_i|x, y) = \frac{p(x, y|c_i)p(c_i)}{p(x, y)}
$$

使用上面这些定义，可以定义贝叶斯分类准则为：

- 如果 $P(c_1|x, y) > P(c_2|x, y)$，那么属于类别 $c_1$；
- 如果 $P(c_2|x, y) > P(c_1|x, y)$，那么属于类别 $c_2$。

**朴素贝叶斯的两个假设**
-   特征之间相互独立
-   每个特征同等重要


In [54]:
import numpy as np

def loadDataSet():
    """
    创建数据集
    :return: 单词列表postingList, 所属类别classVec
    """
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], 
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]  # 1 is abusive, 0 not
    return postingList, classVec

def createVocabSet(dataSet):
    # 求所有list的并集
    return set().union(*dataSet)

def setOfWords2Vec(inputSet, vocabSet):
    return [1 if word in inputSet else 0 for word in vocabSet]

## 从词向量计算概率
首先可以通过类别 i (侮辱性留言或者非侮辱性留言)中的文档数除以总的文档数来计算概率 p(ci) 。接下来计算 p(w | ci) ，这里就要用到朴素贝叶斯假设。如果将 w 展开为一个个独立特征，那么就可以将上述概率写作 p(w0, w1, w2...wn | ci) 。这里假设所有词都互相独立，该假设也称作条件独立性假设，它意味着可以使用 p(w0 | ci)p(w1 | ci)p(w2 | ci)...p(wn | ci) 来计算上述概率，这样就极大地简化了计算的过程。

In [55]:
def _trainNB0(trainMatrix, trainCategory):
    """
    :param trainMatrix: 文件单词矩阵 [[1,0,1,1,1....],[],[]...]
    :param trainCategory: 文件对应的类别[0,1,1,0....]，列表长度等于单词矩阵数，其中的1代表对应的文件是侮辱性文件，0代表不是侮辱性矩阵
    :return:
    """

    # 文件数
    numTrainDocs = len(trainMatrix)
    # 单词数
    numWords = len(trainMatrix[0])
    # 侮辱性文件的出现概率，即trainCategory中所有的1的个数
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    # 构造单词出现次数列表
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)

    # 整个数据集单词出现的次数
    p0Denom = 0.0
    p1Denom = 0.0

    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            # 累加辱骂词的频次
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])

    # 类别1，即侮辱性文档的[P(F1|C1),P(F2|C1),P(F3|C1),P(F4|C1),P(F5|C1)....]列表
    # 即 在1类别下，每个单词出现的概率
    p1Vect = p1Num / p1Denom# [1,2,3,5]/90->[1/90,...]
    # 类别0，即正常文档的[P(F1|C0),P(F2|C0),P(F3|C0),P(F4|C0),P(F5|C0)....]列表
    # 即 在0类别下，每个单词出现的概率
    p0Vect = p0Num / p0Denom
    return p0Vect, p1Vect, pAbusive

## 优化
在利用贝叶斯分类器对文档进行分类时，要计算多个概率的乘积以获得文档属于某个类别的概率，即计算 p(w0|1) * p(w1|1) * p(w2|1)。如果其中一个概率值为 0，那么最后的乘积也为 0。为降低这种影响，可以将所有词的出现数初始化为 1，并将分母初始化为 2 （取1 或 2 的目的主要是为了保证分子和分母不为0，可以根据业务需求进行更改）。

另一个遇到的问题是下溢出，这是由于太多很小的数相乘造成的。当计算乘积 p(w0|ci) * p(w1|ci) * p(w2|ci)... p(wn|ci) 时，由于大部分因子都非常小，所以程序会下溢出或者得到不正确的答案。（用 Python 尝试相乘许多很小的数，最后四舍五入后会得到 0）。一种解决办法是对乘积取自然对数。在代数中有 ln(a * b) = ln(a) + ln(b), 于是通过求对数可以避免下溢出或者浮点数舍入导致的错误。同时，采用自然对数进行处理不会有任何损失。

In [56]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)

    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)

    p0Denom = 2.0
    p1Denom = 2.0

    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i]) 
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])

    p1Vect = np.log(p1Num / p1Denom)
    p0Vect = np.log(p0Num / p0Denom)

    return p0Vect, p1Vect, pAbusive
    

In [57]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    """
    使用算法: 
        # 将乘法转换为加法
        乘法: P(C|F1F2...Fn) = P(F1F2...Fn|C)P(C)/P(F1F2...Fn)
        加法: P(F1|C)*P(F2|C)....P(Fn|C)P(C) -> log(P(F1|C))+log(P(F2|C))+....+log(P(Fn|C))+log(P(C))
    :param vec2Classify: 待测数据[0,1,1,1,1...]，即要分类的向量
    :param p0Vec: 类别0，即正常文档的[log(P(F1|C0)),log(P(F2|C0)),log(P(F3|C0)),log(P(F4|C0)),log(P(F5|C0))....]列表
    :param p1Vec: 类别1，即侮辱性文档的[log(P(F1|C1)),log(P(F2|C1)),log(P(F3|C1)),log(P(F4|C1)),log(P(F5|C1))....]列表
    :param pClass1: 类别1，侮辱性文件的出现概率
    :return: 类别1 or 0
    """
    # 计算公式  log(P(F1|C))+log(P(F2|C))+....+log(P(Fn|C))+log(P(C))
    # 大家可能会发现，上面的计算公式，没有除以贝叶斯准则的公式的分母，也就是 P(w) （P(w) 指的是此文档在所有的文档中出现的概率）就进行概率大小的比较了，
    # 因为 P(w) 针对的是包含侮辱和非侮辱的全部文档，所以 P(w) 是相同的。
    # 使用 NumPy 数组来计算两个向量相乘的结果，这里的相乘是指对应元素相乘，即先将两个向量中的第一个元素相乘，然后将第2个元素相乘，以此类推。
    # 我的理解是: 这里的 vec2Classify * p1Vec 的意思就是将每个词与其对应的概率相关联起来

    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    return 1 if p1 > p0 else 0

def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabSet = createVocabSet(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(set(postinDoc), myVocabSet))
    p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(testEntry, myVocabSet))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))

if __name__ == '__main__':
    testingNB()

['love', 'my', 'dalmation'] classified as:  0


## 实践2：分类邮件

In [58]:
# for i in range(1, 26):
#     f = open(r'D:\Tools\Onedrive\Code\VsCode\Py_Work\Machine_Learning\1\Chapter4\ham\25.txt').read()

In [59]:
# 切分文本
def textParse(bigString):
    import re
    # 使用正则表达式来切分句子，其中分隔符是除单词、数字外的任意字符串
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def spamTest():
    docList = []
    classList = []

    for i in range(1, 26):
        wordList = textParse(open(r'D:\Tools\Onedrive\Code\VsCode\Py_Work\Machine_Learning\1\Chapter4\ham\%d.txt' % i, errors='ignore').read())
        docList.append(wordList)
        classList.append(0)

        wordList = textParse(open(r'D:\Tools\Onedrive\Code\VsCode\Py_Work\Machine_Learning\1\Chapter4\spam\%d.txt' % i, errors='ignore').read())
        docList.append(wordList)
        classList.append(1)

    vocabSet = createVocabSet(docList)

    # 假设 docList 和 classList 已经通过前面的代码构建完成
    import random
    dataIndex = list(range(50))  # 共 50 封邮件，25 封垃圾邮件，25 封非垃圾邮件
    random.shuffle(dataIndex)  # 随机打乱索引

    # 80% 用于训练集，20% 用于测试集
    trainIndex = dataIndex[:40]  # 前 40 个作为训练集
    testIndex = dataIndex[40:]   # 后 10 个作为测试集

    # 构建训练集
    trainMat = []
    trainClasses = []
    for idx in trainIndex:
        trainMat.append(setOfWords2Vec(set(docList[idx]), vocabSet))  # 将词汇列表转换为词向量
        trainClasses.append(classList[idx])  # 对应的标签（0 或 1）

    # 构建测试集
    testMat = []
    testClasses = []
    for idx in testIndex:
        testMat.append(setOfWords2Vec(set(docList[idx]), vocabSet))  # 将词汇列表转换为词向量
        testClasses.append(classList[idx])  # 对应的标签（0 或 1）
    
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    for idx in range(len(testMat)):
        thisDoc = np.array(testMat[idx])
        print(testClasses[idx], 'classified as: ', classifyNB(thisDoc, p0V, p1V, pSpam))
        if testClasses[idx] != classifyNB(thisDoc, p0V, p1V, pSpam):
            print(docList[testIndex[idx]])

if __name__ == '__main__':
    spamTest()



0 classified as:  1
[]
1 classified as:  1
0 classified as:  1
[]
0 classified as:  1
[]
0 classified as:  1
[]
0 classified as:  1
[]
0 classified as:  1
[]
1 classified as:  1
1 classified as:  1
1 classified as:  1
